In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from dataset import SemanticSegmentationDataset
from models.unet import UNet

# define hyperparameters
hyperparameters = {}
hyperparameters['lr'] = 1e-3
hyperparameters['batch_size'] = 1
hyperparameters['epochs'] = 100
hyperparameters['loss'] = 'CE'
# hyperparameters['loss'] = 'Focal'
# hyperparameters['loss'] = 'Dice'
# hyperparameters['loss'] = 'Combo'



# potential categories, choose one set
categories = {}
categories["Class Rarely Or Never Used"] = ["Other Material", "DisturbeView", "VesselLinked", "SolidLargChunk", "PropertiesMaterialInFront", "Other Material"]
categories["Vessel Type Class"] = ["Vessel", "Syringe", "Pippete", "Tube", "IVBag", "DripChamber", "IVBottle", "Beaker", "RoundFlask", "Cylinder", "SeparatoryFunnel", "Funnel", "Burete", "ChromatographyColumn", "Condenser", "Bottle", "Jar", "Connector", "Flask", "Cup", "Bowl", "Erlenmeyer", "Vial", "Dish", "HeatingVessel", "Tube"]
categories["Vessel Properties Class"] = ["Transparent", "SemiTrans", "Opaque", "DisturbeView", "VesselInsideVessel"]
categories['Vessel Part Class'] = ["Cork", "Label", "Part", "Spike", "Valve", "MagneticStirer", "Thermometer", "Spatula", "Holder", "Filter", "PipeTubeStraw"]
categories["Material Property Class"] = ["MaterialOnSurface", "MaterialScattered", "PropertiesMaterialInsideImmersed", "PropertiesMaterialInFront"]
categories["Material Type Class"] =  ["Liquid", "Foam", "Suspension", "Solid", "Powder", "Urine", "Blood", "Gel", "Granular", "SolidLargChunk", "Vapor", "Other Material", "Filled"]
categories_set = "Vessel Part Class"

In [2]:
# data set
train_data = SemanticSegmentationDataset('train', categories=categories[categories_set])
test_data = SemanticSegmentationDataset('test', categories=categories[categories_set])

train_loader = DataLoader(train_data, batch_size=hyperparameters['batch_size'])
test_loader = DataLoader(test_data, batch_size=hyperparameters['batch_size'])

<h2> Loss functions </h2>
<p> Going to run a few experiments with loss functions </p>
<p> 1) Simple cross entropy loss </p>
<p> 2) Class balanced focal loss </p>
<p> 3) Dice loss </p>
<p> 4) Combination of focal loss and dice loss </p>

In [3]:
# init model and opt
# out channels == num classes
model = UNet(in_channels=3, out_channels=len(categories[categories_set]))

optimizer = optim.Adam(model.parameters(), lr=hyperparameters['lr'])
criterion = nn.CrossEntropyLoss()

In [4]:
# one full pass of the shuffled data set with weight updates
def train():
    model.train()
    total_loss = 0.

    for idx, (image, semantic_maps) in enumerate(train_loader):
        # TODO: doublecheck the handling of base and label samples for correct loss calculation
        print(image.shape, semantic_maps.shape)
        assert(0)

        model_output = model(image)

        optimizer.zero_grad()
        loss = criterion(model_output, semantic_maps)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
    
    return total_loss, total_loss / len(train_loader)

In [5]:
# one full pass through the eval set to assess performance
def evaluate():
    model.eval()
    total_loss = 0.

    with torch.no_grad():
        for idx, (image, semantic_maps) in enumerate(test_loader):
            # TODO: doublecheck the handling of base and label samples for correct loss calculation
            model_output = model(image)

            loss = criterion(model_output, semantic_maps)
            total_loss += loss.item()
    
    return total_loss, total_loss / len(train_loader)

In [6]:
# main loop
for e in range(hyperparameters['epochs']):
    train_loss, avg_train_loss = train()
    test_loss, avg_test_loss = evaluate()
    print("Epoch %d Training Loss: %.4f. Validation Loss: %.4f. " % (e, avg_train_loss, avg_test_loss))

c:\Users\jlbar\anaconda3\envs\RL\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([1, 3, 572, 572]) torch.Size([1, 0])


AssertionError: 